# Part 3: Advanced Analysis

In this part, we will implement advanced analysis techniques for physiological time series data, including time-domain feature extraction, frequency analysis, and wavelet transforms.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import pywt

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_context('notebook')

## 1. Time-Domain Feature Extraction

Implement the `extract_time_domain_features` function to extract various time-domain features from physiological signals.

In [11]:
def extract_time_domain_features(data, window_size=60):
    """Extract time-domain features from physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    window_size : int, optional
        Size of the rolling window in seconds, default=60
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing extracted features for each window
    """
    # Convert window_size from seconds to number of samples
    # Assuming data is sampled at 1 Hz (1 sample per second)
    window_samples = window_size
    
    # Initialize DataFrame for features
    features = pd.DataFrame()
    
    # Basic statistics using rolling window
    features['mean'] = data['heart_rate'].rolling(window=window_samples).mean()
    features['std'] = data['heart_rate'].rolling(window=window_samples).std()
    features['min'] = data['heart_rate'].rolling(window=window_samples).min()
    features['max'] = data['heart_rate'].rolling(window=window_samples).max()
    
    # Heart rate statistics
    features['mean_hr'] = data['heart_rate'].rolling(window=window_samples).mean()
    features['std_hr'] = data['heart_rate'].rolling(window=window_samples).std()
    
    # Beat-to-beat variability
    rr_intervals = 60000 / data['heart_rate']  # Convert HR to RR intervals in ms
    
    # Calculate successive differences within each window
    rr_diff = rr_intervals.diff()
    
    # RMSSD (Root Mean Square of Successive Differences)
    features['rmssd'] = np.sqrt(rr_diff.rolling(window=window_samples).apply(lambda x: np.mean(x**2)))
    
    # SDNN (Standard Deviation of NN intervals)
    features['sdnn'] = rr_intervals.rolling(window=window_samples).std()
    
    # pNN50 (Percentage of successive RR intervals differing by >50ms)
    features['pnn50'] = rr_diff.rolling(window=window_samples).apply(
        lambda x: 100 * np.sum(np.abs(x) > 50) / len(x) if len(x) > 0 else 0
    )
    
    # Drop NaN values from rolling window calculations
    features = features.dropna()
    
    return features

## 2. Frequency Analysis

Implement the `analyze_frequency_components` function to perform frequency-domain analysis on the signals.

In [12]:
def analyze_frequency_components(data, sampling_rate, window_size=60):
    """Perform frequency-domain analysis on physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
        
    Returns:
    --------
    dict
        Dictionary containing frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_frequencies = []
    all_power = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Calculate PSD using Welch's method
        frequencies, power = signal.welch(
            window_data,
            fs=sampling_rate,
            nperseg=window_samples
        )
        
        all_frequencies.append(frequencies)
        all_power.append(power)
    
    # Average results across windows
    results['frequencies'] = np.mean(all_frequencies, axis=0)
    results['power'] = np.mean(all_power, axis=0)
    
    # Define frequency bands
    bands = {
        'VLF': (0.003, 0.04),
        'LF': (0.04, 0.15),
        'HF': (0.15, 0.4)
    }
    
    # Calculate power in each band
    results['bands'] = {}
    for band_name, (low, high) in bands.items():
        mask = (results['frequencies'] >= low) & (results['frequencies'] <= high)
        results['bands'][band_name] = np.sum(results['power'][mask])
    
    # Calculate LF/HF ratio
    results['bands']['LF/HF'] = results['bands']['LF'] / results['bands']['HF']
    
    return results

## 3. Time-Frequency Analysis

Implement the `analyze_time_frequency_features` function to analyze time-frequency features using wavelet transforms.

In [13]:
def analyze_time_frequency_features(data, sampling_rate, window_size=60):
    """Analyze time-frequency features using wavelet transforms.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
        
    Returns:
    --------
    dict
        Dictionary containing time-frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Define wavelet scales
    scales = np.arange(1, 128)
    results['scales'] = scales
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_coefficients = []
    all_energy = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Apply continuous wavelet transform
        coefficients, frequencies = pywt.cwt(
            window_data,
            scales,
            'morl',
            sampling_period=1/sampling_rate
        )
        
        all_coefficients.append(coefficients)
        all_energy.append(np.abs(coefficients)**2)
    
    # Average results across windows
    results['coefficients'] = np.mean(all_coefficients, axis=0)
    results['time_frequency_energy'] = np.mean(all_energy, axis=0)
    
    return results

## Example Usage

Here's how to use these functions with your data:

In [14]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2


In [ ]:
# First, load the processed data
import pandas as pd
import glob
import os
import numpy as np

# Load all processed data files
files = glob.glob('data/processed/*_processed.csv')
dfs = [pd.read_csv(f) for f in files]
processed_data = pd.concat(dfs, ignore_index=True)

# Make sure timestamp is datetime
processed_data['timestamp'] = pd.to_datetime(processed_data['timestamp'])

# Get unique subjects and sessions
subjects = processed_data['subject_id'].unique()
sessions = processed_data['session'].unique()

# Create output directory
os.makedirs('data/advanced_analysis', exist_ok=True)

# Process each subject and session
for subject in subjects:
    for session in sessions:
        print(f"Processing {subject}, {session}")  # Optional: to track progress
        
        # Filter data for this subject/session
        mask = (processed_data['subject_id'] == subject) & (processed_data['session'] == session)
        subject_data = processed_data[mask]
        
        if len(subject_data) == 0:
            continue
            
        # 1. Extract time domain features
        time_features = extract_time_domain_features(subject_data, window_size=60)
        time_features.to_csv(f'data/advanced_analysis/{subject}_{session}_time_features.csv')
        
        # 2. Analyze frequency components
        sampling_rate = 4.0  # Hz
        freq_results = analyze_frequency_components(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_fft.npy', freq_results)
        
        # 3. Analyze time-frequency features
        tf_results = analyze_time_frequency_features(subject_data, sampling_rate, window_size=60)
        np.save(f'data/advanced_analysis/{subject}_{session}_wavelet.npy', tf_results)

Processing S1, Final
Processing S1, Midterm 1
Processing S1, Midterm 2
Processing S10, Final
Processing S10, Midterm 1
Processing S10, Midterm 2
Processing S2, Final
Processing S2, Midterm 1
Processing S2, Midterm 2
Processing S3, Final
Processing S3, Midterm 1
Processing S3, Midterm 2
Processing S4, Final
Processing S4, Midterm 1
Processing S4, Midterm 2
Processing S5, Final
Processing S5, Midterm 1
Processing S5, Midterm 2
Processing S6, Final
Processing S6, Midterm 1
Processing S6, Midterm 2
Processing S7, Final
Processing S7, Midterm 1
Processing S7, Midterm 2
Processing S8, Final
Processing S8, Midterm 1
Processing S8, Midterm 2
Processing S9, Final
Processing S9, Midterm 1
Processing S9, Midterm 2
